In [1]:
import torch
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
import spacy
nlp = spacy.load('en_core_web_trf')

from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(ngram_range=(2,3))

import nltk
stopwords = set(nltk.corpus.stopwords.words('english'))

In [3]:
from helpers.cloze_generation import generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.table_processing import preprocess_table, read_process_table, find_relevant_column_header, find_relevant_content, linearize_table
from helpers.t5_language_model import summarise_t5_results
from helpers.configs import CLOZE_MASKS

df = pd.read_pickle('pickles/collection_20210624_194932.pkl')
# clozes_df = pd.read_json('pickles/clozes_20210824.json')
clozes_df = pd.read_json('pickles/clozes_with_false_20210825.json') #contains negs

clozes_df['answer_type_agg'] = clozes_df['answer_type'].apply(
    lambda x: CLOZE_MASKS.get(x))

usabilities = pd.read_json('secondary/dataframe_usability_3.json')

# Example Amphetamines

In [4]:
where = ['Amphetamine use' in x for x in clozes_df.cloze_text.values]
selection = clozes_df.iloc[where]

In [5]:
df_iterator = read_process_table(selection.data.values[0][0])

In [6]:
table = next(df_iterator)
table = next(df_iterator)
table = next(df_iterator)
table 

,England and Wales Any cocaine,England and Wales Jan '95 to Dec '95,England and Wales Jan '97 to Dec '97,England and Wales Jan '99 to Dec '99,England and Wales Apr '01 to Mar '02,England and Wales Apr '02 to Mar '03,England and Wales Apr '03 to Mar '04,England and Wales Apr '04 to Mar '05,England and Wales Apr '05 to Mar '06,England and Wales Apr '06 to Mar '07,...,England and Wales Apr '13 to Mar '14,England and Wales Apr '14 to Mar '15,England and Wales Apr '15 to Mar '16,England and Wales Apr '16 to Mar '17,England and Wales Apr '17 to Mar '18,England and Wales Apr '18 to Mar '19,England and Wales Apr '19 to Mar '20,England and Wales Jan '95 to Dec '95,England and Wales Apr '09 to \nMar '10,Adults aged 16 to 59 Apr '18 to \nMar '19
0,Any cocaine,3.1139518,3.8069302,5.59720904,5.19090443,6.1161763,6.75106039,6.17825149,7.36851382,7.80495801,...,9.45175646,9.71240348,9.74656936,9.66771305,10.68537929,10.76847052,10.619746957109626,*,*,nan
1,Powder cocaine,2.97283206,3.67042094,5.47814185,5.07108022,5.99681428,6.6285372,6.07765419,7.25110533,7.66102968,...,9.35411306,9.63984965,9.65434582,9.60209276,10.61115766,10.68265066,10.540796963125098,*,*,nan
2,Crack cocaine,0.72133501,0.72065817,1.05737459,0.68864182,0.86438163,0.8830419,0.77170081,0.85894156,0.96550611,...,1.02234418,1.15705641,1.00173452,0.85916393,0.80881023,0.83348315,0.7021160296197497,*,*,nan
3,Ecstasy,3.80947169,4.21612702,5.34799617,5.84914688,6.57082308,6.80169978,6.75365728,7.31710616,7.41965848,...,9.27928917,9.17474662,9.38182555,8.95612403,10.04779789,9.87837249,9.560718048089631,*,*,nan
4,Hallucinogens,7.79421814,8.38410671,9.33913606,8.19936371,9.12523506,9.37114354,8.57840212,9.48344509,9.21352891,...,9.05636021,8.45589172,8.49402044,8.22769068,9.11167201,8.52825464,8.396783945921785,*,*,nan
5,LSD,5.42107812,5.61855983,6.18594593,5.36813896,5.83511446,6.04357261,5.2195329,5.56561937,5.47136806,...,5.26395015,4.69063392,4.42351522,4.60918391,5.20585805,4.95390291,4.660301289839254,*,*,nan
6,Magic mushrooms,5.3250938,5.96013218,7.00014228,6.06046868,6.69197182,7.00769386,6.57490603,7.38216363,7.23646934,...,7.3426916,7.09818186,7.06071922,6.7413985,7.31797476,6.90325346,6.762005699760951,*,*,nan
7,Opiates,0.74461601,0.87235145,1.07154913,0.68994727,0.93989473,0.95811337,0.77112434,0.87045454,0.81855923,...,1.11222881,0.94435235,0.80643095,0.71329245,0.89615525,0.67352997,0.5962328525176348,*,*,nan
8,Heroin,0.63029463,0.59866718,0.99007589,0.5821161,0.75632944,0.83461058,0.64469043,0.65116889,0.72369547,...,0.82544139,0.68179831,0.58292972,0.54710382,0.63744781,0.52616742,0.4621546724433781,*,*,nan
9,Methadone,0.31863018,0.47302639,0.45546513,0.37091677,0.44294112,0.36459936,0.43121881,0.47573001,0.36612669,...,0.55620889,0.56635296,0.43764935,0.37836664,0.44479861,0.39484805,0.34962050835419284,*,*,nan


In [7]:
from transformers import pipeline

unmasker = pipeline('fill-mask', model = 'bert-large-uncased')

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_tf_idf_query_similarity(allDocs, query, ngram_range=(1,3)):
    vectorizer = TfidfVectorizer(ngram_range=ngram_range)
    # allDocs = []
    # for document in documents:
    #     allDocs.append(nlp.clean_tf_idf_text(document))
    docTFIDF = vectorizer.fit_transform(allDocs)
    queryTFIDF = vectorizer.fit(allDocs)
    queryTFIDF = queryTFIDF.transform([query])

    cosineSimilarities = cosine_similarity(queryTFIDF, docTFIDF).flatten()
    return cosineSimilarities

In [9]:
cloze = selection.cloze_text.values[0].replace('TEMPORALMASK', '[MASK]')
linearized = linearize_table(table)

/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/pafitis/miniforge3/envs/comp0087/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
_input = cloze + linearized

In [16]:
unmasker(cloze, truncation = True, top_k = 1)

[{'sequence': 'amphetamine use in australia in adults aged 16 to 59 years fell by 42 % compared with the previous year ( to 109, 000 people ), continuing the long - term decline since the year ending december 1995.',
  'score': 0.09185995161533356,
  'token': 2660,
  'token_str': 'australia'}]

In [15]:
unmasker(_input, truncation = True, top_k = 1)

[{'sequence': "amphetamine use in wales in adults aged 16 to 59 years fell by 42 % compared with the previous year ( to 109, 000 people ), continuing the long - term decline since the year ending december 1995. row 1 : england and wales any cocaine is any cocaine ; england and wales jan'95 to dec'95 is 3. 1139518 ; england and wales jan'97 to dec'97 is 3. 8069302 ; england and wales jan'99 to dec'99 is 5. 59720904 ; england and wales apr'01 to mar'02 is 5. 19090443 ; england and wales apr'02 to mar'03 is 6. 1161763 ; england and wales apr'03 to mar'04 is 6. 75106039 ; england and wales apr'04 to mar'05 is 6. 17825149 ; england and wales apr'05 to mar'06 is 7. 36851382 ; england and wales apr'06 to mar'07 is 7. 80495801 ; england and wales apr'07 to mar'08 is 7. 75929345 ; england and wales apr'08 to mar'09 is 9. 31177403 ; england and wales apr'09 to mar'10 is 8. 692329302186957 ; england and wales apr'10 to mar'11 is 8. 90701303 ; england and wales apr'11 to mar'12 is 9. 59076549 ; en